In [114]:
# Author: QiangZiBro
# Time: 2021-10-14
# Contact: Github/QiangZiBro

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
# MacOS上画图中文乱码的问题
# https://blog.csdn.net/minixuezhen/article/details/81516949?utm_medium=distribute.pc_relevant.none-task-blog-
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

# %matplotlib inline

import sys
import os

from pprint import pprint

if not os.path.exists("results"):
    os.mkdir("results")
    
FIGSIZE = (30, 25)
FORMAT = "jpg"

## 数据加载工具

In [179]:
PLACES = ['A', 'B', 'C', 'A1', 'A2', 'A3']
TYPES = [0, 1, 2]
def _load_data():
    table_files = """data/附件1 监测点A空气质量预报基础数据.xlsx
data/附件2 监测点B、C空气质量预报基础数据.xlsx
data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx""".split("\n")
    tables = [pd.read_excel("../"+i, engine='openpyxl', sheet_name=None) for i in table_files] # 表格全读取
    keys = [list(i.keys()) for i in tables] # 每个表格的sheet list
    return tables, keys

def _process_to_json(tables):
    sheetnames = {0:'监测点{}逐小时污染物浓度与气象一次预报数据', 1:'监测点{}逐小时污染物浓度与气象实测数据', 2:'监测点{}逐日污染物浓度实测数据'}
    result = {i:{} for i in PLACES}
    
    for k,v in sheetnames.items():
        tables[0][v.format('A')].name = v.format('A')
        result['A'][k] = tables[0][v.format('A')]
    for c in ['B', 'C']:
        for k,v in sheetnames.items():
            tables[1][v.format(c)].name = v.format(c)
            result[c][k] = tables[1][v.format(c)]
    for c in ['A1', 'A2', 'A3']:
        for k,v in sheetnames.items():
            tables[2][v.format(c)].name = v.format(c)
            result[c][k] = tables[2][v.format(c)]
    return result

def load(): 
    """得到一个Json格式的数据项
    
    Return:
        {
            "A":{
                "0":DataFrame,
                "1":DataFrame,
                "2":DataFrame,
            }
            ...
        }
    """
    
    tables, keys = _load_data()
    return _process_to_json(tables)

data = load()

In [180]:
print(data.keys())
print(data['A'].keys())
# for p in PLACES:
#     for i in range(3): 
#         print(data[p][i].min())

dict_keys(['A', 'B', 'C', 'A1', 'A2', 'A3'])
dict_keys([0, 1, 2])


In [157]:
df = data['B'][1]
df.isna().sum()

监测时间                  0
地点                    0
SO2监测浓度(μg/m³)      248
NO2监测浓度(μg/m³)      275
PM10监测浓度(μg/m³)     285
PM2.5监测浓度(μg/m³)    276
O3监测浓度(μg/m³)       203
CO监测浓度(mg/m³)       270
温度(℃)                56
湿度(%)                56
气压(MBar)             56
近地风速(m/s)            56
风向(°)                56
dtype: int64

In [181]:
num = df._get_numeric_data()
num[num<0] = np.nan
num.head()

,SO2监测浓度(μg/m³),NO2监测浓度(μg/m³),PM10监测浓度(μg/m³),PM2.5监测浓度(μg/m³),O3监测浓度(μg/m³),CO监测浓度(mg/m³),温度(℃),湿度(%),气压(MBar),近地风速(m/s),风向(°)
0,5.0,29.0,60.0,25.0,116.0,0.5,20.9,84.0,1014.7,2.0,140.1
1,5.0,30.0,65.0,22.0,113.0,0.5,20.9,82.0,1014.0,1.9,139.6
2,5.0,23.0,57.0,23.0,117.0,0.5,20.7,82.0,1013.7,1.6,126.4
3,6.0,17.0,46.0,23.0,114.0,0.5,20.3,82.0,1013.1,0.9,61.8
4,5.0,23.0,42.0,26.0,102.0,0.5,18.3,94.0,1012.6,1.1,138.0


In [184]:
num.isna().sum()

SO2监测浓度(μg/m³)      248
NO2监测浓度(μg/m³)      275
PM10监测浓度(μg/m³)     285
PM2.5监测浓度(μg/m³)    276
O3监测浓度(μg/m³)       203
CO监测浓度(mg/m³)       270
温度(℃)                56
湿度(%)                56
气压(MBar)             56
近地风速(m/s)            56
风向(°)                56
dtype: int64

## 预处理完分析

In [209]:
import sys
sys.path.append("..")
from common.io import load_table
df = load_table("A")


InvalidFileException: openpyxl does not support  file format, please check you can open it with Excel first. Supported formats are: .xlsx,.xlsm,.xltx,.xltm

In [207]:
df['监测点A1逐日污染物浓度实测数据'].isna().sum()

Unnamed: 0                0
监测日期                      0
地点                        0
SO2监测浓度(μg/m³)            0
NO2监测浓度(μg/m³)            0
PM10监测浓度(μg/m³)           0
PM2.5监测浓度(μg/m³)          0
O3最大八小时滑动平均监测浓度(μg/m³)    0
CO监测浓度(mg/m³)             0
dtype: int64

In [206]:
df['监测点A1逐日污染物浓度实测数据'] = df['监测点A1逐日污染物浓度实测数据'].fillna(df['监测点A1逐日污染物浓度实测数据'].mean())

<ipython-input-206-f44bc60b3c91>:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df['监测点A1逐日污染物浓度实测数据'] = df['监测点A1逐日污染物浓度实测数据'].fillna(df['监测点A1逐日污染物浓度实测数据'].mean())


## 空值分析

In [117]:
def null_values_analysis(data):
    NULL_COUNTS = {}
    for p in PLACES:
        for t in TYPES:  
            if data[p][t].isnull().sum().sum():
                NULL_COUNTS.update({p+"_"+str(t):data[p][t].isnull().sum()})
    return NULL_COUNTS

def null_values_analysis_plot(null_table):
    fig, axs = plt.subplots(3, 3, figsize=FIGSIZE)
    fig.tight_layout(h_pad=10)
    fig.suptitle('空值分析', size=20)
    
    for i, (k,v) in enumerate(null_table.items()):
        axs[i//3][i%3].bar(v.keys(), v, 0.35)
        axs[i//3][i%3].set_title(k)
        axs[i//3][i%3].set_xticklabels(labels=v.keys(),rotation=45)
        axs[i//3][i%3].grid(True)

    font = {'family': 'serif',
            'color':  'darkred',
            'weight': 'normal',
            'size': 16,
            }

    plt.savefig("results/nan_value_analysis."+FORMAT, format=FORMAT)
    plt.close()
null_table = null_values_analysis(data)
null_values_analysis_plot(null_table)

<ipython-input-117-5fff5f5b585c>:17: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i//3][i%3].set_xticklabels(labels=v.keys(),rotation=45)


# 数据分析
下面首先对各量随时间的分布进行分析
注意数字和子表格的对应：
- 0 逐小时污染物浓度与气象一次预报数据
- 1 逐小时污染物浓度与气象实测数据
- 2 逐日污染物浓度实测数据

## 0 对逐小时污染物浓度与气象一次预报数据的分析

In [118]:
key = data['A'][0].columns
key, len(key[3:])

(Index(['模型运行日期', '预测时间', '地点', '近地2米温度（℃）', '地表温度（K）', '比湿（kg/kg）', '湿度（%）',
        '近地10米风速（m/s）', '近地10米风向（°）', '雨量（mm）', '云量', '边界层高度（m）', '大气压（Kpa）',
        '感热通量（W/m²）', '潜热通量（W/m²）', '长波辐射（W/m²）', '短波辐射（W/m²）', '地面太阳能辐射（W/m²）',
        'SO2小时平均浓度(μg/m³)', 'NO2小时平均浓度(μg/m³)', 'PM10小时平均浓度(μg/m³)',
        'PM2.5小时平均浓度(μg/m³)', 'O3小时平均浓度(μg/m³)', 'CO小时平均浓度(mg/m³)'],
       dtype='object'),
 21)

In [119]:
PREDICTED_KEYS = key[3:]
print("预测天气状况",PREDICTED_KEYS[:-6])
print("预测污染因素",PREDICTED_KEYS[-6:])

预测天气状况 Index(['近地2米温度（℃）', '地表温度（K）', '比湿（kg/kg）', '湿度（%）', '近地10米风速（m/s）',
       '近地10米风向（°）', '雨量（mm）', '云量', '边界层高度（m）', '大气压（Kpa）', '感热通量（W/m²）',
       '潜热通量（W/m²）', '长波辐射（W/m²）', '短波辐射（W/m²）', '地面太阳能辐射（W/m²）'],
      dtype='object')
预测污染因素 Index(['SO2小时平均浓度(μg/m³)', 'NO2小时平均浓度(μg/m³)', 'PM10小时平均浓度(μg/m³)',
       'PM2.5小时平均浓度(μg/m³)', 'O3小时平均浓度(μg/m³)', 'CO小时平均浓度(mg/m³)'],
      dtype='object')


In [120]:
def plot_prediction_hist_by_time(place):
    "对逐小时污染物浓度与气象一次预报数据的分析"
    key = data[place][0].columns
    PREDICTED_KEYS = key[3:]
    fig, axes = plt.subplots(nrows=7, ncols=3, figsize=FIGSIZE)
    fig.tight_layout(h_pad=3)
    plt.suptitle(place+"_0", size=30)
    
    for i, k in enumerate(PREDICTED_KEYS):
        data[place][0].plot.line(ax = axes[i//3][i%3], x = key[1] , y=k)
    plt.savefig("results/{}_0逐小时污染物浓度与气象一次预报数据.{}".format(place, FORMAT), format=FORMAT)
    plt.close()


for p in PLACES:
    plot_prediction_hist_by_time(p)

## 1 对逐小时污染物浓度与气象实测数据的分析

In [121]:
key = data['A'][1].columns
key, len(key[2:])

(Index(['监测时间', '地点', 'SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)',
        'PM2.5监测浓度(μg/m³)', 'O3监测浓度(μg/m³)', 'CO监测浓度(mg/m³)', '温度(℃)', '湿度(%)',
        '气压(MBar)', '风速(m/s)', '风向(°)'],
       dtype='object'),
 11)

In [122]:
MEASURED_KEYS = key[2:]
print("测量天气状况",MEASURED_KEYS[:6])
print("测量污染因素",MEASURED_KEYS[6:])

测量天气状况 Index(['SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)',
       'PM2.5监测浓度(μg/m³)', 'O3监测浓度(μg/m³)', 'CO监测浓度(mg/m³)'],
      dtype='object')
测量污染因素 Index(['温度(℃)', '湿度(%)', '气压(MBar)', '风速(m/s)', '风向(°)'], dtype='object')


In [123]:
def plot_measured_hist_by_hour(place):
    "对逐小时污染物浓度与气象一次预报数据的分析"
    key = data[place][1].columns
    MEASURED_KEYS = key[2:]
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=FIGSIZE)
    for i, k in enumerate(MEASURED_KEYS):
        axes[i//3][i%3].set_xticks([])
        # 预处理
        data[place][1][k].fillna(0)
        # 将object类型转为数值类型
        data[place][1][k] = pd.to_numeric(data[place][1][k], errors='coerce')
        data[place][1].plot.line(ax = axes[i//3][i%3], x = key[0] , y=k)
    fig.tight_layout(h_pad=3)
    plt.suptitle(place+"_1", size=30)
    plt.savefig("results/{}_1逐小时污染物浓度与气象实测数据.{}".format(place, FORMAT), format=FORMAT)
    plt.close()
    
for p in PLACES:
    plot_measured_hist_by_hour(p)

## 2 对逐日污染物浓度实测数据的分析

In [124]:
key = data['A'][2].columns
key, len(key[2:])

(Index(['监测日期', '地点', 'SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)',
        'PM2.5监测浓度(μg/m³)', 'O3最大八小时滑动平均监测浓度(μg/m³)', 'CO监测浓度(mg/m³)'],
       dtype='object'),
 6)

In [125]:
MEASURED_KEYS = key[2:]
print("测量污染因素",MEASURED_KEYS[:])

测量污染因素 Index(['SO2监测浓度(μg/m³)', 'NO2监测浓度(μg/m³)', 'PM10监测浓度(μg/m³)',
       'PM2.5监测浓度(μg/m³)', 'O3最大八小时滑动平均监测浓度(μg/m³)', 'CO监测浓度(mg/m³)'],
      dtype='object')


In [126]:
def plot_measured_polution_hist_by_day(place):
    "对逐小时污染物浓度与气象一次预报数据的分析"
    key = data[place][2].columns
    MEASURED_KEYS = key[2:]
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=FIGSIZE)
    for i, k in enumerate(MEASURED_KEYS):
        # 预处理
        data[place][2][k].fillna(0)
        # 将object类型转为数值类型
        data[place][2][k] = pd.to_numeric(data[place][2][k], errors='coerce')
        data[place][2].plot.line(ax = axes[i//3][i%3], x = key[0] , y=k)
    fig.tight_layout(h_pad=3)
    plt.suptitle(place+"_2", size=30)
    plt.savefig("results/{}_2逐日污染物浓度实测数据.{}".format(place, FORMAT), format=FORMAT)
    plt.close()
# 解锁运行  
for p in PLACES:
    plot_measured_polution_hist_by_day(p)